In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib as plt
%matplotlib inline

import sqlite3 as db

In [2]:
# https://towardsdatascience.com/an-easy-beginners-guide-to-sqlite-in-python-and-pandas-fbf1f38f6800
# so the line of code above will connect to a file called my_database.db IF IT EXISTS in the folder. If it does not exist, a file called my_database.db will automatically be created on your computer 

conn = db.connect('../data/news_popularity.db')

# In order to interact with our database, we now have to create a cursor. This is what we will use to issue commands that will allow us to query or modify our database.
# c = conn.cursor()
# good practise to close if not using
# c.close()
# conn.close()

In [3]:
# simplest way
# df = pd.read_sql_query('select * from sqlite_master', conn)
# df

cursor = conn.cursor()
cursor.execute("SELECT name from sqlite_master WHERE type='table';")

print(cursor.fetchall())
cursor.close()

[('articles',), ('description',), ('keywords',)]


In [4]:
df_articles = pd.read_sql_query('select * from articles',conn)
df_description = pd.read_sql_query('select * from description', conn)
df_keywords = pd.read_sql_query('select * from keywords',conn)

In [5]:
print(df_articles.shape)
df_articles.head()

(39644, 5)


,ID,url,timedelta,weekday,shares
0,0,http://mashable.com/2013/01/07/amazon-instant-...,731.0,monday,593
1,1,http://mashable.com/2013/01/07/ap-samsung-spon...,731.0,monday,711
2,2,http://mashable.com/2013/01/07/apple-40-billio...,731.0,monday,1500
3,3,http://mashable.com/2013/01/07/astronaut-notre...,731.0,monday,1200
4,4,http://mashable.com/2013/01/07/att-u-verse-apps/,731.0,monday,505


In [6]:
print(df_description.shape)
df_description.head()

(39644, 16)


,ID,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,n_comments,average_token_length,data_channel,self_reference_min_shares,self_reference_max_shares,self_reference_avg_shares
0,0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,0.0,0.0,4.680365,entertainment,496.0,496.0,496.000000
1,1,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,NaN,28.0,4.913725,business,0.0,0.0,0.000000
2,2,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,0.0,15.0,4.393365,business,918.0,918.0,918.000000
3,3,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,0.0,72.0,4.404896,entertainment,0.0,0.0,0.000000
4,4,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,NaN,20.0,4.682836,technology,545.0,16000.0,3151.157895


In [7]:
print(df_keywords.shape)
df_keywords.head()

(35680, 11)


,ID,num_keywords,kw_min_min,kw_max_min,kw_avg_min,kw_min_max,kw_max_max,kw_avg_max,kw_min_avg,kw_max_avg,kw_avg_avg
0,0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### so it looks like they have common ID, and we can assume we can join them together for more context, but keywords seem to have lesser row for some reason?

In [8]:
df_keywords.tail()

,ID,num_keywords,kw_min_min,kw_max_min,kw_avg_min,kw_min_max,kw_max_max,kw_avg_max,kw_min_avg,kw_max_avg,kw_avg_avg
35675,39638,6.0,-1.0,459.0,91.000,0.0,843300.0,484083.333333,0.000000,4301.332394,2665.713159
35676,39639,8.0,-1.0,671.0,173.125,26900.0,843300.0,374962.500000,2514.742857,4004.342857,3031.115764
35677,39640,7.0,-1.0,616.0,184.000,6500.0,843300.0,192985.714286,1664.267857,5470.168651,3411.660830
35678,39642,5.0,-1.0,0.0,-1.000,0.0,843300.0,254600.000000,0.000000,3384.316871,1777.895883
35679,39643,4.0,-1.0,97.0,23.500,205600.0,843300.0,366200.000000,3035.080555,3613.512953,3296.909481


In [55]:
df_keywords['ID'].count()

35680

In [9]:
df_articles.tail()

,ID,url,timedelta,weekday,shares
39639,39639,http://mashable.com/2014/12/27/samsung-app-aut...,8.0,wednesday,1800
39640,39640,http://mashable.com/2014/12/27/seth-rogen-jame...,8.0,wednesday,1900
39641,39641,http://mashable.com/2014/12/27/son-pays-off-mo...,8.0,wednesday,1900
39642,39642,http://mashable.com/2014/12/27/ukraine-blasts/,8.0,wednesday,1100
39643,39643,http://mashable.com/2014/12/27/youtube-channel...,8.0,wednesday,1300


In [56]:
df_articles['ID'].count()

39644

In [47]:
df_description.tail()

,ID,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,n_comments,average_token_length,data_channel,self_reference_min_shares,self_reference_max_shares,self_reference_avg_shares
39639,39639,11.0,346.0,0.529052,1.0,0.684783,9.0,7.0,1.0,NaN,72.0,4.523121,technology,11400.0,48000.0,37033.333333
39640,39640,12.0,328.0,0.696296,1.0,0.885057,9.0,7.0,3.0,48.0,76.0,4.405488,social_media,2100.0,2100.0,2100.000000
39641,39641,10.0,442.0,0.516355,1.0,0.644128,24.0,1.0,12.0,1.0,0.0,5.076923,None,1400.0,1400.0,1400.000000
39642,39642,6.0,682.0,0.539493,1.0,0.692661,10.0,1.0,1.0,0.0,0.0,4.975073,world,452.0,452.0,452.000000
39643,39643,10.0,157.0,0.701987,1.0,0.846154,1.0,1.0,0.0,NaN,130.0,4.471338,entertainment,2100.0,2100.0,2100.000000


In [57]:
df_description['ID'].isnull().count()

39644

In [10]:
df_articles['ID'].is_unique

True

In [11]:
df_description['ID'].is_unique

True

In [12]:
df_keywords['ID'].is_unique

True

### data here will suggest that keywords have lesser and as such, the other columns have to be dropped.
this  is cause


1) we can generate mean or median or random values, except that will not be possible for url, therefore i feel it is best to drop the extra rows.

2) url is unique and not duplicated

3) ID is also unique

In [27]:
rowdiff = 39644 - 35680
print('difference in row = ', rowdiff)

difference in row =  3964


### SQL joins
https://www.analyticsvidhya.com/blog/2021/06/join-the-dataframes-like-sql-tables-in-python-using-pandas/


(INNER) JOIN: Returns only those records that have matching values in both DataFrames

LEFT (OUTER) JOIN: Returns all the records from the left DataFrame and the matched records from the right DataFrame

RIGHT (OUTER) JOIN: Returns all the records from the right DataFrame, and the matched records from the left DataFrame

FULL (OUTER) JOIN: Returns all the records when there is a match in either left or right DataFrame


https://www.analyticsvidhya.com/blog/2021/06/15-pandas-functions-to-replicate-basic-sql-queries-in-python/

In [63]:
# https://pandas.pydata.org/docs/user_guide/merging.html

# Something to note is that whilst the default for merge() is "inner", the default for join() is left. 
# So if you are working with index joins be mindful. 

df_premerge = pd.merge(df_keywords, df_description,  on="ID",how="outer")
df_merged = pd.merge(df_premerge, df_articles, on="ID",how="outer")

In [64]:
df_merged.head()

,ID,num_keywords,kw_min_min,kw_max_min,kw_avg_min,kw_min_max,kw_max_max,kw_avg_max,kw_min_avg,kw_max_avg,...,n_comments,average_token_length,data_channel,self_reference_min_shares,self_reference_max_shares,self_reference_avg_shares,url,timedelta,weekday,shares
0,0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,4.680365,entertainment,496.0,496.0,496.000000,http://mashable.com/2013/01/07/amazon-instant-...,731.0,monday,593
1,1,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,28.0,4.913725,business,0.0,0.0,0.000000,http://mashable.com/2013/01/07/ap-samsung-spon...,731.0,monday,711
2,2,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,15.0,4.393365,business,918.0,918.0,918.000000,http://mashable.com/2013/01/07/apple-40-billio...,731.0,monday,1500
3,3,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,72.0,4.404896,entertainment,0.0,0.0,0.000000,http://mashable.com/2013/01/07/astronaut-notre...,731.0,monday,1200
4,4,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,20.0,4.682836,technology,545.0,16000.0,3151.157895,http://mashable.com/2013/01/07/att-u-verse-apps/,731.0,monday,505


In [65]:
df_merged.shape

(39644, 30)

In [66]:
df_merged.describe()

,ID,num_keywords,kw_min_min,kw_max_min,kw_avg_min,kw_min_max,kw_max_max,kw_avg_max,kw_min_avg,kw_max_avg,...,num_self_hrefs,num_imgs,num_videos,n_comments,average_token_length,self_reference_min_shares,self_reference_max_shares,self_reference_avg_shares,timedelta,shares
count,39644.000000,35680.000000,35680.000000,35680.000000,35680.000000,35680.000000,35680.000000,35680.000000,35680.000000,35680.000000,...,38851.000000,38058.000000,21011.000000,39644.000000,39644.000000,38851.000000,38851.000000,38851.000000,39644.000000,39644.000000
mean,19821.500000,7.220095,26.169703,1142.296851,310.297441,13533.498851,752318.562220,259143.787814,1115.670961,5637.977838,...,3.294458,4.538336,1.284565,94.971168,4.548239,4007.686775,10357.140660,6417.144920,354.530471,3395.380184
std,11444.381373,1.909604,69.705812,3844.829034,616.961660,57623.779316,214391.922961,135080.424060,1136.741358,6026.965362,...,3.864212,8.285066,4.144714,472.550267,0.844406,19876.663280,41340.581032,24390.372003,214.163767,11626.950749
min,0.000000,1.000000,-1.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,1.000000
25%,9910.750000,6.000000,-1.000000,445.000000,141.440476,0.000000,843300.000000,172595.650000,0.000000,3559.646658,...,1.000000,1.000000,0.000000,2.000000,4.478404,638.000000,1100.000000,980.000000,164.000000,946.000000
50%,19821.500000,7.000000,-1.000000,658.000000,235.428571,1400.000000,843300.000000,244133.333333,1020.667532,4348.584651,...,3.000000,1.000000,0.000000,16.000000,4.664082,1200.000000,2800.000000,2200.000000,339.000000,1400.000000
75%,29732.250000,9.000000,4.000000,1000.000000,357.000000,7800.000000,843300.000000,330980.000000,2058.093423,6015.434738,...,4.000000,4.000000,1.000000,78.000000,4.854839,2600.000000,8000.000000,5200.000000,542.000000,2800.000000
max,39643.000000,10.000000,377.000000,298400.000000,42827.857143,843300.000000,843300.000000,843300.000000,3610.124972,298400.000000,...,116.000000,128.000000,91.000000,61790.000000,8.041534,843300.000000,843300.000000,843300.000000,731.000000,843300.000000


In [67]:
df_merged.dtypes

ID                             int64
num_keywords                 float64
kw_min_min                   float64
kw_max_min                   float64
kw_avg_min                   float64
kw_min_max                   float64
kw_max_max                   float64
kw_avg_max                   float64
kw_min_avg                   float64
kw_max_avg                   float64
kw_avg_avg                   float64
n_tokens_title               float64
n_tokens_content             float64
n_unique_tokens              float64
n_non_stop_words             float64
n_non_stop_unique_tokens     float64
num_hrefs                    float64
num_self_hrefs               float64
num_imgs                     float64
num_videos                   float64
n_comments                   float64
average_token_length         float64
data_channel                  object
self_reference_min_shares    float64
self_reference_max_shares    float64
self_reference_avg_shares    float64
url                           object
t

### findings
nice to see most of them are numerical, lesser work and conversion to do..

In [69]:
df_merged.isnull().sum()

ID                               0
num_keywords                  3964
kw_min_min                    3964
kw_max_min                    3964
kw_avg_min                    3964
kw_min_max                    3964
kw_max_max                    3964
kw_avg_max                    3964
kw_min_avg                    3964
kw_max_avg                    3964
kw_avg_avg                    3964
n_tokens_title                   0
n_tokens_content                 0
n_unique_tokens                  0
n_non_stop_words                 0
n_non_stop_unique_tokens         0
num_hrefs                      793
num_self_hrefs                 793
num_imgs                      1586
num_videos                   18633
n_comments                       0
average_token_length             0
data_channel                  6134
self_reference_min_shares      793
self_reference_max_shares      793
self_reference_avg_shares      793
url                              0
timedelta                        0
weekday             

so there are some null values to handle..

In [70]:
df_merged.tail()

,ID,num_keywords,kw_min_min,kw_max_min,kw_avg_min,kw_min_max,kw_max_max,kw_avg_max,kw_min_avg,kw_max_avg,...,n_comments,average_token_length,data_channel,self_reference_min_shares,self_reference_max_shares,self_reference_avg_shares,url,timedelta,weekday,shares
39639,39591,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,4.631579,world,839.0,2800.0,1746.333333,http://mashable.com/2014/12/26/de-blasio-ramos...,9.0,tuesday,1100
39640,39597,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,168.0,4.791339,technology,1400.0,3300.0,2125.000000,http://mashable.com/2014/12/26/hilarious-news-...,9.0,tuesday,2100
39641,39623,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,95.0,4.146789,technology,878.0,4200.0,1776.600000,http://mashable.com/2014/12/26/what-is-tor/,9.0,tuesday,947
39642,39628,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,40.0,5.078275,world,921.0,5800.0,3360.500000,http://mashable.com/2014/12/27/facebook-year-i...,9.0,tuesday,1000
39643,39641,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,5.076923,None,1400.0,1400.0,1400.000000,http://mashable.com/2014/12/27/son-pays-off-mo...,8.0,wednesday,1900


In [71]:
df_merged

,ID,num_keywords,kw_min_min,kw_max_min,kw_avg_min,kw_min_max,kw_max_max,kw_avg_max,kw_min_avg,kw_max_avg,...,n_comments,average_token_length,data_channel,self_reference_min_shares,self_reference_max_shares,self_reference_avg_shares,url,timedelta,weekday,shares
0,0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,4.680365,entertainment,496.0,496.0,496.000000,http://mashable.com/2013/01/07/amazon-instant-...,731.0,monday,593
1,1,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,28.0,4.913725,business,0.0,0.0,0.000000,http://mashable.com/2013/01/07/ap-samsung-spon...,731.0,monday,711
2,2,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,15.0,4.393365,business,918.0,918.0,918.000000,http://mashable.com/2013/01/07/apple-40-billio...,731.0,monday,1500
3,3,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,72.0,4.404896,entertainment,0.0,0.0,0.000000,http://mashable.com/2013/01/07/astronaut-notre...,731.0,monday,1200
4,4,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,20.0,4.682836,technology,545.0,16000.0,3151.157895,http://mashable.com/2013/01/07/att-u-verse-apps/,731.0,monday,505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39639,39591,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,4.631579,world,839.0,2800.0,1746.333333,http://mashable.com/2014/12/26/de-blasio-ramos...,9.0,tuesday,1100
39640,39597,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,168.0,4.791339,technology,1400.0,3300.0,2125.000000,http://mashable.com/2014/12/26/hilarious-news-...,9.0,tuesday,2100
39641,39623,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,95.0,4.146789,technology,878.0,4200.0,1776.600000,http://mashable.com/2014/12/26/what-is-tor/,9.0,tuesday,947
39642,39628,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,40.0,5.078275,world,921.0,5800.0,3360.500000,http://mashable.com/2014/12/27/facebook-year-i...,9.0,tuesday,1000


In [75]:
df_merge_clean = df_merged.dropna(subset=['num_keywords','kw_min_min','kw_max_min','kw_avg_min'])

In [76]:
df_merge_clean.shape

(35680, 30)

In [77]:
df_merge_clean.describe()

,ID,num_keywords,kw_min_min,kw_max_min,kw_avg_min,kw_min_max,kw_max_max,kw_avg_max,kw_min_avg,kw_max_avg,...,num_self_hrefs,num_imgs,num_videos,n_comments,average_token_length,self_reference_min_shares,self_reference_max_shares,self_reference_avg_shares,timedelta,shares
count,35680.000000,35680.000000,35680.000000,35680.000000,35680.000000,35680.000000,35680.000000,35680.000000,35680.000000,35680.000000,...,34959.000000,34261.000000,18925.000000,35680.000000,35680.000000,34959.000000,34959.000000,34959.000000,35680.000000,35680.000000
mean,19823.468918,7.220095,26.169703,1142.296851,310.297441,13533.498851,752318.562220,259143.787814,1115.670961,5637.977838,...,3.282531,4.513003,1.286446,94.122001,4.547799,4026.946198,10308.613255,6424.933389,354.523234,3402.054961
std,11452.292291,1.909604,69.705812,3844.829034,616.961660,57623.779316,214391.922961,135080.424060,1136.741358,6026.965362,...,3.834065,8.259735,4.148189,476.050347,0.843522,20110.819102,40998.053595,24588.092091,214.301086,11934.878541
min,0.000000,1.000000,-1.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,4.000000
25%,9890.750000,6.000000,-1.000000,445.000000,141.440476,0.000000,843300.000000,172595.650000,0.000000,3559.646658,...,1.000000,1.000000,0.000000,2.000000,4.477958,638.000000,1100.000000,978.000000,163.000000,946.000000
50%,19801.500000,7.000000,-1.000000,658.000000,235.428571,1400.000000,843300.000000,244133.333333,1020.667532,4348.584651,...,3.000000,1.000000,0.000000,16.000000,4.663366,1200.000000,2800.000000,2200.000000,339.000000,1400.000000
75%,29788.250000,9.000000,4.000000,1000.000000,357.000000,7800.000000,843300.000000,330980.000000,2058.093423,6015.434738,...,4.000000,4.000000,1.000000,78.000000,4.853695,2600.000000,7900.000000,5166.666667,543.000000,2800.000000
max,39643.000000,10.000000,377.000000,298400.000000,42827.857143,843300.000000,843300.000000,843300.000000,3610.124972,298400.000000,...,116.000000,128.000000,91.000000,61790.000000,8.041534,843300.000000,843300.000000,843300.000000,731.000000,843300.000000


In [78]:
df_merge_clean.dtypes

ID                             int64
num_keywords                 float64
kw_min_min                   float64
kw_max_min                   float64
kw_avg_min                   float64
kw_min_max                   float64
kw_max_max                   float64
kw_avg_max                   float64
kw_min_avg                   float64
kw_max_avg                   float64
kw_avg_avg                   float64
n_tokens_title               float64
n_tokens_content             float64
n_unique_tokens              float64
n_non_stop_words             float64
n_non_stop_unique_tokens     float64
num_hrefs                    float64
num_self_hrefs               float64
num_imgs                     float64
num_videos                   float64
n_comments                   float64
average_token_length         float64
data_channel                  object
self_reference_min_shares    float64
self_reference_max_shares    float64
self_reference_avg_shares    float64
url                           object
t